In [1]:
import os
os.chdir("../../../../")

import pandas as pd
import numpy as np
import chardet

In [2]:
avid_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviw_path = os.getcwd() + "/data/tourism/rank_psg_data.csv"

In [3]:
with open(aviw_path, 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}


In [4]:
aviw = pd.read_csv(aviw_path, sep="\t", encoding="utf-16")
aviw.columns = [col.lower().replace(" ", "_") for col in aviw.columns]
aviw["date"] = pd.to_datetime(aviw["day_of_date"])
aviw.head(5)

,day_of_date,destination_country,psg_wow_change,psg_yoy_change,passengers_number_7days,date
0,"January 1, 2020",Tonga,-42.97%,-2.83%,2544,2020-01-01
1,"January 2, 2020",Tonga,-40.05%,-7.19%,2593,2020-01-02
2,"January 3, 2020",Tonga,-19.66%,3.80%,2894,2020-01-03
3,"January 4, 2020",Tonga,-22.05%,-18.70%,2704,2020-01-04
4,"January 5, 2020",Tonga,-1.09%,-6.45%,2986,2020-01-05


In [5]:
avid = pd.read_excel(avid_path)
avid.columns = [col.lower().replace(" ", "_") for col in avid.columns]
avid["date"] = pd.to_datetime(avid["date"])
avid.head(5)

,country,iso,region,date,aircraft_type,seats_arrivals_domestic,seats_arrivals_interregional,seats_arrivals_intraregional,seats_arrivals_intl,seats_arrivals_total,available_seat_kilometers,number_of_flights_domestic,number_of_flights_interregional,number_of_flights_intraregional,number_of_flights_intl,number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [6]:
countries = aviw.destination_country.unique().tolist()
for con in ["Tonga"]:
    df = (aviw[aviw.destination_country == con].reset_index().drop(
        ["index", "day_of_date", "psg_wow_change", "psg_yoy_change", "destination_country"], axis=1))
    df_ref = (avid[avid.country == con].reset_index().drop("index", axis=1))
    date_range = pd.DataFrame(
        pd.date_range(start="2020-01-01", end=df["date"].iloc[-1], freq="D"), 
     columns=["date"])
    df = date_range.merge(df, how="left", on="date").fillna(0)
    df_ref = date_range.merge(df_ref, how="left", on="date").fillna(0)
    df_ref["6days_sum_from_daily"] = df_ref["seats_arrivals_intl"].rolling(6).sum()
    df = df.merge(df_ref[["date", "seats_arrivals_intl", "6days_sum_from_daily"]],
                  how="inner", on="date")

In [7]:
df["7days_sum_from_daily"] = df["seats_arrivals_intl"].rolling(7).sum()
df["diff"] = abs(df["6days_sum_from_daily"] - df["7days_sum_from_daily"])
df["calc_psg"] = df["diff"].shift(-6)